In [1]:
# Import libraries and data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

full_train = pd.read_parquet('../data/train/all_features_train.parquet')
full_test = pd.read_parquet('../data/test/all_features_test.parquet')

In [2]:
X_train, y_train = (full_train.drop('price', axis = 1), full_train.price)
X_test, y_test = (full_test.drop('price', axis = 1), full_test.price)

In [8]:
X_train.columns

Index(['address', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'greenbelt', 'nuisance',
       ...
       'log(sqft_basement_plus_1)_x_grade_Low_Average',
       'log(sqft_basement_plus_1)_x_grade_Luxury',
       'log(sqft_basement_plus_1)_x_grade_Poor',
       'log(sqft_basement_plus_1)_x_grade_Very_Good',
       'log(sqft_basement_plus_1)_x_log(sqft_above_plus_0)',
       'log(sqft_basement_plus_1)_x_log(sqft_lot_plus_0)',
       'log(sqft_basement_plus_1)_x_log(sqft_living_plus_0)',
       'log(sqft_basement_plus_1)_x_log(price_plus_0)',
       'log(sqft_basement_plus_1)_x_log(sqft_patio_plus_1)',
       'log(sqft_basement_plus_1)_x_log(sqft_garage_plus_1)'],
      dtype='object', length=15880)

In [ ]:
full_train.corr().abs().price.nlargest(5)

In [ ]:
simple = ['sqft_living']

In [3]:
import time

In [ ]:
%%time

lr = LinearRegression()
lr.fit(X_train.drop(['address', 'date'], axis = 1), y_train)
lr.score(X_test.drop(['address', 'date'], axis = 1), y_test)